link to colab: https://colab.research.google.com/drive/1ZEVqkM_iN28SxPC4Q9zcRfnpGHUg3A4R?usp=sharing

### shots_df
Удари матчу, 27 штук  
Цікаві фічі: `minute`, `(x, y)`, `xG`, `result` ('Goal', 'ShotOnPost', 'MissedShots', 'BlockedShot', 'SavedShot'), `situation` ('OpenPlay', 'FromCorner', 'SetPiece'), `shotType` ('RightFoot', 'LeftFoot', 'Head'), `player_assisted`, `lastAction` ('None', 'BallRecovery', 'Pass', 'Chipped', 'Cross', 'Rebound', 'Dispossessed', 'Foul', 'BallTouch')  
можна ще дістати історію матчів сезону або всіх матчів попередніх 10 сезонів

### players_df
Гравці матчу з обох сторін. Є інфа про удари, голи, ключові паси, позицію, зіграний час на полі, червоні/жовті картки,  `xA`, `xG`, `xGBuildup`,  `xGChain` (що б це не означало :D)

### player_matches_history_df
Те саме, що players, тільки вся історія гравця, можна відфільтрувати тільки 2023 сезон

### player_shots_history_df
Те саме, що shots, тільки вся історія гравця

### player_avg_stats_df
Статистика гравця за останні 10 сезонів (мін, макс, середнє значення для кожного сезону)

### team_res_history_df
Історія матчів команди за останні 10 сезонів (результат, xG, ставки)

### team_stats_history_df
Статистика команди за останні 10 сезонів. Багато різної інфи типу attackSpeed, formation, але це все для цілого сезону

In [2]:
#!pip install soccerdata
!pip install understat asyncio aiohttp > /dev/null

In [5]:
#import soccerdata as sd
import asyncio
import aiohttp
from understat import Understat

import pprint
import json
import pandas as pd

In [6]:
# ws = sd.WhoScored(leagues="ESP-La Liga")
# print(ws.__doc__)


In [7]:
# ws = sd.WhoScored(leagues="ESP-La Liga", seasons=2024)

In [8]:
def to_df(table):
  df = pd.DataFrame(table)
  df.columns = df.iloc[0]
  df = df[1:]
  return df

In [9]:
async with aiohttp.ClientSession() as session:
    understat = Understat(session)
    table = await understat.get_league_table("la_liga", "2023")
    league_df = to_df(table)

league_df

ClientConnectorError: Cannot connect to host understat.com:443 ssl:default [nodename nor servname provided, or not known]

In [14]:
async with aiohttp.ClientSession() as session:
    understat = Understat(session)
    results = await understat.get_league_results(
        "la_liga",
        2023,
        {
            "a": {"id": "150",
                  "title": "Real Madrid",
                  "short_title": "RMA"},
            "h": {"id": "148",
                  "title": "Barcelona",
                  "short_title": "BAR"}
        }
    )

pprint.pprint(results)

[{'a': {'id': '150', 'short_title': 'RMA', 'title': 'Real Madrid'},
  'datetime': '2023-10-28 14:15:00',
  'forecast': {'d': '0.3183', 'l': '0.2342', 'w': '0.4475'},
  'goals': {'a': '2', 'h': '1'},
  'h': {'id': '148', 'short_title': 'BAR', 'title': 'Barcelona'},
  'id': '22787',
  'isResult': True,
  'xG': {'a': '0.959404', 'h': '1.32038'}}]


In [20]:
h = 'Barcelona'
a = 'Real Madrid'

In [31]:
async with aiohttp.ClientSession() as session:
    understat = Understat(session)
    shots = await understat.get_match_shots(22787)
    #pprint.pprint(shots)

# print example
shots['a'][0]

{'id': '545509',
 'minute': '12',
 'result': 'BlockedShot',
 'X': '0.8109999847412109',
 'Y': '0.5020000076293946',
 'xG': '0.05580964311957359',
 'player': 'Vinícius Júnior',
 'h_a': 'a',
 'player_id': '7008',
 'situation': 'OpenPlay',
 'season': '2023',
 'shotType': 'RightFoot',
 'match_id': '22787',
 'h_team': 'Barcelona',
 'a_team': 'Real Madrid',
 'h_goals': '1',
 'a_goals': '2',
 'date': '2023-10-28 14:15:00',
 'player_assisted': 'Rodrygo',
 'lastAction': 'Pass'}

In [32]:
shots_df = pd.concat([pd.DataFrame(shots['h']), pd.DataFrame(shots['a'])]).reset_index()
print(shots_df.columns)
shots_df.head(5)

Index(['index', 'id', 'minute', 'result', 'X', 'Y', 'xG', 'player', 'h_a',
       'player_id', 'situation', 'season', 'shotType', 'match_id', 'h_team',
       'a_team', 'h_goals', 'a_goals', 'date', 'player_assisted',
       'lastAction'],
      dtype='object')


,index,id,minute,result,X,Y,xG,player,h_a,player_id,...,season,shotType,match_id,h_team,a_team,h_goals,a_goals,date,player_assisted,lastAction
0,0,545508,5,Goal,0.9259999847412109,0.5359999847412109,0.548054039478302,Ilkay Gündogan,h,314,...,2023,RightFoot,22787,Barcelona,Real Madrid,1,2,2023-10-28 14:15:00,None,None
1,1,545510,15,ShotOnPost,0.9069999694824219,0.6309999847412109,0.09907364845275879,Fermín López,h,11822,...,2023,RightFoot,22787,Barcelona,Real Madrid,1,2,2023-10-28 14:15:00,Gavi,BallRecovery
2,2,545513,36,MissedShots,0.779000015258789,0.4879999923706055,0.040702082216739655,Fermín López,h,11822,...,2023,LeftFoot,22787,Barcelona,Real Madrid,1,2,2023-10-28 14:15:00,Ferrán Torres,Pass
3,3,545515,41,BlockedShot,0.924000015258789,0.6719999694824219,0.06672355532646179,João Félix,h,7892,...,2023,LeftFoot,22787,Barcelona,Real Madrid,1,2,2023-10-28 14:15:00,João Cancelo,Chipped
4,4,545517,45,MissedShots,0.8119999694824219,0.485,0.04046473652124405,Gavi,h,9804,...,2023,RightFoot,22787,Barcelona,Real Madrid,1,2,2023-10-28 14:15:00,None,None


In [165]:
async with aiohttp.ClientSession() as session:
    understat = Understat(session)
    players = await understat.get_match_players(22787)
    pprint.pprint(players)

{'a': {'619943': {'assists': '0',
                  'goals': '0',
                  'h_a': 'a',
                  'id': '619943',
                  'key_passes': '0',
                  'own_goals': '0',
                  'player': 'Kepa',
                  'player_id': '5061',
                  'position': 'GK',
                  'positionOrder': '1',
                  'red_card': '0',
                  'roster_in': '0',
                  'roster_out': '0',
                  'shots': '0',
                  'team_id': '150',
                  'time': '90',
                  'xA': '0',
                  'xG': '0',
                  'xGBuildup': '0.023411521688103676',
                  'xGChain': '0.023411521688103676',
                  'yellow_card': '0'},
       '619944': {'assists': '0',
                  'goals': '0',
                  'h_a': 'a',
                  'id': '619944',
                  'key_passes': '0',
                  'own_goals': '0',
                  'player': 'D

In [37]:
players_list = []
for team in players.values():
  for player in team.values():
    players_list.append(player)

players_df = pd.DataFrame(players_list)
players_df

,id,goals,own_goals,shots,xG,time,player_id,team_id,position,player,...,yellow_card,red_card,roster_in,roster_out,key_passes,assists,xA,xGChain,xGBuildup,positionOrder
0,619928,0,0,0,0,90,2090,148,GK,Marc-André ter Stegen,...,0,0,0,0,0,0,0,0.02325536124408245,0.02325536124408245,1
1,619931,0,0,1,0.07281102985143661,90,8091,148,DC,Ronald Araújo,...,0,0,0,0,0,0,0,0,0,3
2,619929,0,0,0,0,90,200,148,DC,Andreas Christensen,...,0,0,0,0,0,0,0,0.04295586794614792,0.04295586794614792,3
3,619930,0,0,1,0.09640434384346008,90,2484,148,DC,Iñigo Martínez,...,0,0,0,0,0,0,0,0.10742563754320145,0.10742563754320145,3
4,619932,0,0,1,0.0977792888879776,79,2379,148,MR,João Cancelo,...,0,0,619942,0,1,0,0.06672355532646179,0.20520491898059845,0.040702082216739655,8
5,619934,0,0,2,0.08342060446739197,90,9804,148,MC,Gavi,...,0,0,0,0,1,0,0.09907364845275879,0.1420295238494873,0,9
6,619933,1,0,1,0.548054039478302,90,314,148,MC,Ilkay Gündogan,...,0,0,0,0,0,0,0,0.715732753276825,0.16767875850200653,9
7,619935,0,0,0,0,90,9805,148,ML,Álex Balde,...,0,0,0,0,0,0,0,0,0,10
8,619936,0,0,2,0.13977573812007904,74,11822,148,AMC,Fermín López,...,1,0,619941,0,1,0,0.09640434384346008,0.13977573812007904,0,12
9,619937,0,0,1,0.06672355532646179,79,7892,148,FW,João Félix,...,0,0,619939,0,0,0,0,0.06672355532646179,0,15


In [108]:
player_matches_history_list = []
player_dict = players_df.set_index('player_id')['player'].to_dict()
player_ids = players_df['player_id'].values

async with aiohttp.ClientSession() as session:
    understat = Understat(session)
        # Using **kwargs
    for id in player_ids:
      player_matches = await understat.get_player_matches(
          id) #, h_team="Barcelona", season="2023")
      player_matches_df = pd.DataFrame(player_matches)
      player_matches_df['player_id'] = id
      player_matches_df['player'] = player_dict[id]
      player_matches_history_list.append(player_matches_df)

player_matches_history_df = pd.concat(player_matches_history_list)

# filter out games after our target match
player_matches_history_df['date'] = pd.to_datetime(player_matches_history_df['date'])
player_matches_history_df = player_matches_history_df[player_matches_history_df['date'] <= pd.Timestamp('2023-10-28')]
player_matches_history_df

,goals,shots,xG,time,position,h_team,a_team,h_goals,a_goals,date,...,roster_id,xA,assists,key_passes,npg,npxG,xGChain,xGBuildup,player_id,player
2,0,0,0,90,GK,Barcelona,Real Madrid,1,2,2023-10-28,...,619928,0,0,0,0,0,0.02325536124408245,0.02325536124408245,2090,Marc-André ter Stegen
3,0,0,0,90,GK,Barcelona,Athletic Club,1,0,2023-10-22,...,619182,0,0,0,0,0,0.18332354724407196,0.18332354724407196,2090,Marc-André ter Stegen
4,0,0,0,90,GK,Granada,Barcelona,2,2,2023-10-08,...,617585,0,0,0,0,0,0,0,2090,Marc-André ter Stegen
5,0,0,0,90,GK,Barcelona,Sevilla,1,0,2023-09-29,...,614167,0,0,0,0,0,1.4326690435409546,1.4326690435409546,2090,Marc-André ter Stegen
6,0,0,0,90,GK,Mallorca,Barcelona,2,2,2023-09-26,...,613526,0,0,0,0,0,0.0513811931014061,0.0513811931014061,2090,Marc-André ter Stegen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,1,2,0.1521420031785965,90,FW,Hannover 96,FC Cologne,1,0,2014-09-24,...,28600,0.1165580004453659,0,2,1,0.1521420031785965,0.1915159970521927,0.039374299347400665,866,Joselu
277,0,2,0.07242730259895325,77,FW,Paderborn,Hannover 96,2,0,2014-09-20,...,28446,0.01334839966148138,0,1,0,0.07242730259895325,0.11229799687862396,0.07423459738492966,866,Joselu
278,0,3,0.4962660074234009,90,FW,Hannover 96,Hamburger SV,2,0,2014-09-14,...,28097,0.9664710164070129,1,1,0,0.4962660074234009,1.9906100034713745,0.5638359785079956,866,Joselu
279,0,4,0.41295599937438965,90,FW,Mainz 05,Hannover 96,0,0,2014-08-31,...,27806,0.025337200611829758,0,1,0,0.41295599937438965,0.45803800225257874,0.019745200872421265,866,Joselu


In [109]:
# get data for 2023 season
player_matches_history_df[player_matches_history_df['season'] == '2023']

,goals,shots,xG,time,position,h_team,a_team,h_goals,a_goals,date,...,roster_id,xA,assists,key_passes,npg,npxG,xGChain,xGBuildup,player_id,player
2,0,0,0,90,GK,Barcelona,Real Madrid,1,2,2023-10-28,...,619928,0,0,0,0,0,0.02325536124408245,0.02325536124408245,2090,Marc-André ter Stegen
3,0,0,0,90,GK,Barcelona,Athletic Club,1,0,2023-10-22,...,619182,0,0,0,0,0,0.18332354724407196,0.18332354724407196,2090,Marc-André ter Stegen
4,0,0,0,90,GK,Granada,Barcelona,2,2,2023-10-08,...,617585,0,0,0,0,0,0,0,2090,Marc-André ter Stegen
5,0,0,0,90,GK,Barcelona,Sevilla,1,0,2023-09-29,...,614167,0,0,0,0,0,1.4326690435409546,1.4326690435409546,2090,Marc-André ter Stegen
6,0,0,0,90,GK,Mallorca,Barcelona,2,2,2023-09-26,...,613526,0,0,0,0,0,0.0513811931014061,0.0513811931014061,2090,Marc-André ter Stegen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,1,4,0.9662458896636963,64,FW,Real Madrid,Real Sociedad,2,1,2023-09-17,...,611499,0,0,0,1,0.9662458896636963,1.0156375169754028,0.04939166456460953,866,Joselu
15,1,5,1.1646673679351807,90,FW,Real Madrid,Getafe,2,1,2023-09-02,...,608779,0.112339548766613,0,1,1,1.1646673679351807,0.8234151005744934,0.1706867665052414,866,Joselu
16,0,0,0,73,Sub,Celta Vigo,Real Madrid,0,1,2023-08-25,...,606640,0.46505823731422424,1,1,0,0,0,0,866,Joselu
17,0,1,0.45651039481163025,2,Sub,Almeria,Real Madrid,1,3,2023-08-19,...,605459,0,0,0,0,0.45651039481163025,0.9412375688552856,0.4847272038459778,866,Joselu


In [107]:
player_shots_history_list = []

async with aiohttp.ClientSession() as session:
    understat = Understat(session)
        # Using **kwargs
    for id in player_ids:
      player_shots = await understat.get_player_shots(id)
      player_shots_history_list += player_shots

player_shots_history_df = pd.DataFrame(player_shots_history_list)

# filter out games after our target match
player_shots_history_df['date'] = pd.to_datetime(player_shots_history_df['date'])
player_shots_history_df = player_shots_history_df[player_shots_history_df['date'] <= pd.Timestamp('2023-10-28 23:30:00')]
player_shots_history_df

,id,minute,result,X,Y,xG,player,h_a,player_id,situation,season,shotType,match_id,h_team,a_team,h_goals,a_goals,date,player_assisted,lastAction
0,414126,93,MissedShots,0.8330000305175781,0.49,0.05627699941396713,Marc-André ter Stegen,a,2090,SetPiece,2020,RightFoot,15074,Real Madrid,Barcelona,2,1,2021-04-10 19:00:00,None,None
1,364352,59,ShotOnPost,0.9769999694824218,0.4129999923706055,0.39333000779151917,Ronald Araújo,a,8091,FromCorner,2019,RightFoot,12297,Mallorca,Barcelona,0,4,2020-06-13 20:00:00,Lionel Messi,Pass
2,393514,13,BlockedShot,0.909000015258789,0.43099998474121093,0.061559099704027176,Ronald Araújo,h,8091,FromCorner,2020,Head,14905,Barcelona,Real Sociedad,2,1,2020-12-16 20:00:00,Lionel Messi,Cross
3,394016,51,Goal,0.8919999694824219,0.575999984741211,0.08176220208406448,Ronald Araújo,h,8091,OpenPlay,2020,RightFoot,14909,Barcelona,Valencia,2,2,2020-12-19 15:15:00,None,None
4,395960,21,SavedShot,0.919000015258789,0.35700000762939454,0.019022399559617043,Ronald Araújo,h,8091,FromCorner,2020,Head,14928,Barcelona,Eibar,1,1,2020-12-29 18:00:00,Miralem Pjanic,Cross
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6636,541446,70,SavedShot,0.8419999694824218,0.37099998474121093,0.10520801693201065,Joselu,a,866,OpenPlay,2023,RightFoot,22760,Girona,Real Madrid,0,3,2023-09-30 16:30:00,Aurelien Tchouameni,Pass
6637,542740,46,MissedShots,0.9069999694824219,0.6220000076293946,0.08969507366418839,Joselu,h,866,OpenPlay,2023,LeftFoot,22772,Real Madrid,Osasuna,4,0,2023-10-07 14:15:00,Luka Modric,Chipped
6638,542747,69,Goal,0.8930000305175781,0.4209999847412109,0.4971727728843689,Joselu,h,866,OpenPlay,2023,RightFoot,22772,Real Madrid,Osasuna,4,0,2023-10-07 14:15:00,Vinícius Júnior,Throughball
6639,542752,83,SavedShot,0.885,0.5,0.7432776093482971,Joselu,h,866,Penalty,2023,RightFoot,22772,Real Madrid,Osasuna,4,0,2023-10-07 14:15:00,None,Standard


In [128]:
player_avg_stats_list = []

async with aiohttp.ClientSession() as session:
    understat = Understat(session)
        # Using **kwargs
    for id in player_ids:
      player_avg_info = await understat.get_player_stats(id)
      #pprint.pprint(player_avg_info)
      player_stats_df = pd.DataFrame(pd.json_normalize(player_avg_info))
      player_stats_df['player_id'] = id
      player_stats_df['player'] = player_dict[id]
      player_avg_stats_list.append(player_stats_df)

player_avg_stats_df = pd.concat(player_avg_stats_list)

# filter out games after our target match
player_avg_stats_df

,position,goals.min,goals.max,goals.avg,xG.min,xG.max,xG.avg,shots.min,shots.max,shots.avg,...,key_passes.max,key_passes.avg,xGChain.min,xGChain.max,xGChain.avg,xGBuildup.min,xGBuildup.max,xGBuildup.avg,player_id,player
0,GK,0.0000,0.0001,0.0000,0.000000,0.000071,0.000001,0.0000,0.0003,0.0000,...,0.0014,0.0003,0.000052,0.003064,0.000855,0.000049,0.003064,0.000841,2090,Marc-André ter Stegen
1,Sub,0.0008,0.0160,0.0026,0.001212,0.014437,0.002744,0.0089,0.0660,0.0249,...,0.0369,0.0162,0.003784,0.016049,0.005900,0.000342,0.008153,0.002759,2090,Marc-André ter Stegen
0,DC,0.0000,0.0021,0.0005,0.000028,0.001892,0.000494,0.0006,0.0171,0.0050,...,0.0089,0.0021,0.000434,0.006843,0.001888,0.000380,0.006696,0.001771,8091,Ronald Araújo
1,DL,0.0000,0.0023,0.0003,0.000008,0.002380,0.000341,0.0005,0.0180,0.0058,...,0.0199,0.0088,0.000959,0.008929,0.002787,0.000802,0.007058,0.002169,8091,Ronald Araújo
2,DR,0.0000,0.0017,0.0003,0.000006,0.001722,0.000337,0.0002,0.0149,0.0052,...,0.0210,0.0082,0.001072,0.008822,0.002823,0.000681,0.007371,0.002188,8091,Ronald Araújo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,MC,0.0000,0.0052,0.0009,0.000033,0.003700,0.000922,0.0012,0.0354,0.0124,...,0.0377,0.0115,0.000854,0.012126,0.003672,0.000642,0.008059,0.002569,7359,Eduardo Camavinga
3,Sub,0.0008,0.0160,0.0026,0.001212,0.014437,0.002744,0.0089,0.0660,0.0249,...,0.0369,0.0162,0.003784,0.016049,0.005900,0.000342,0.008153,0.002759,7359,Eduardo Camavinga
0,AMC,0.0003,0.0062,0.0023,0.000660,0.004983,0.002192,0.0111,0.0425,0.0218,...,0.0451,0.0186,0.002750,0.010071,0.004880,0.001072,0.006206,0.002221,866,Joselu
1,FW,0.0011,0.0126,0.0042,0.001728,0.012082,0.004155,0.0150,0.0737,0.0280,...,0.0309,0.0120,0.002727,0.016914,0.005338,0.000243,0.006713,0.001318,866,Joselu


In [149]:
import numpy as np
team_res_history_list = []

async with aiohttp.ClientSession() as session:
    understat = Understat(session)
    for season in np.arange(2014, 2024, 1):
      team_res = await understat.get_team_results("Real Madrid", season=season)
      df = pd.DataFrame(pd.json_normalize(team_res))
      team_res_history_list.append(df)

team_res_history_df = pd.concat(team_res_history_list)
team_res_history_df['datetime'] = pd.to_datetime(team_res_history_df['datetime'])
team_res_history_df = team_res_history_df[team_res_history_df['datetime'] <= pd.Timestamp('2023-10-28 23:30:00')]
team_res_history_df

,id,isResult,side,datetime,result,h.id,h.title,h.short_title,a.id,a.title,a.short_title,goals.h,goals.a,xG.h,xG.a,forecast.w,forecast.d,forecast.l
0,5834,True,h,2014-08-25 19:00:00,w,150,Real Madrid,RMA,206,Cordoba,COR,2,0,0.612645,0.37841,0.347663,0.462355,0.189983
1,5844,True,a,2014-08-31 20:00:00,l,140,Real Sociedad,SOC,150,Real Madrid,RMA,4,2,2.74099,2.31574,0.483372,0.179541,0.337087
2,5789,True,h,2014-09-13 19:00:00,l,150,Real Madrid,RMA,143,Atletico Madrid,ATL,1,2,1.58778,1.11054,0.484062,0.249813,0.266125
3,5797,True,a,2014-09-20 15:00:00,w,139,Deportivo La Coruna,DEP,150,Real Madrid,RMA,2,8,1.74836,2.05644,0.334134,0.210491,0.455375
4,5806,True,h,2014-09-23 19:00:00,w,150,Real Madrid,RMA,207,Elche,ELC,5,1,3.93785,1.07447,0.861622,0.083016,0.055345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,22753,True,h,2023-09-27 17:00:00,w,150,Real Madrid,RMA,144,Las Palmas,LPL,2,0,2.71542,0.618032,0.813680,0.125855,0.060464
7,22760,True,a,2023-09-30 16:30:00,w,223,Girona,GIR,150,Real Madrid,RMA,0,3,2.17781,2.78602,0.305913,0.178206,0.515881
8,22772,True,h,2023-10-07 14:15:00,w,150,Real Madrid,RMA,157,Osasuna,OSA,4,0,4.47802,0.164111,0.979386,0.018132,0.002412
9,22782,True,a,2023-10-21 16:30:00,d,138,Sevilla,SEV,150,Real Madrid,RMA,1,1,1.8723,2.17904,0.339607,0.203509,0.456884


note: forcast is probably for home team, not the query team

In [156]:
team_stats_history_list = []

async with aiohttp.ClientSession() as session:
    understat = Understat(session)
    for season in np.arange(2014, 2024, 1):
      team_stats = await understat.get_team_stats("Real Madrid", season=season)
      df = pd.DataFrame(pd.json_normalize(team_stats))
      df['season'] = season
      team_stats_history_list.append(df)

team_stats_history_df = pd.concat(team_stats_history_list)
display(team_stats_history_df)

print("\ncolumns:")
_= [print(x) for x in team_stats_history_df.columns]

,situation.OpenPlay.shots,situation.OpenPlay.goals,situation.OpenPlay.xG,situation.OpenPlay.against.shots,situation.OpenPlay.against.goals,situation.OpenPlay.against.xG,situation.FromCorner.shots,situation.FromCorner.goals,situation.FromCorner.xG,situation.FromCorner.against.shots,...,formation.3-4-1-2.against.goals,formation.3-4-1-2.against.xG,formation.4-2-4-0.stat,formation.4-2-4-0.time,formation.4-2-4-0.shots,formation.4-2-4-0.goals,formation.4-2-4-0.xG,formation.4-2-4-0.against.shots,formation.4-2-4-0.against.goals,formation.4-2-4-0.against.xG
0,495,89,74.394665,334,26,29.170928,101,11,8.405605,65,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,550,85,73.908812,327,25,33.565995,100,13,8.365774,52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,505,78,69.715487,307,33,27.972420,83,11,8.386906,51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,544,67,73.262978,303,33,34.917615,101,15,11.026905,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,442,43,52.891116,286,34,34.480113,99,9,8.173679,56,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,424,49,52.829878,237,18,23.301811,91,4,7.206471,52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,403,51,52.466107,270,15,25.959797,97,8,7.961815,44,...,0.0,0.162761,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,520,61,61.402428,321,23,34.262375,79,10,6.999489,56,...,NaN,NaN,4-2-4-0,47.0,4.0,0.0,0.163093,11.0,3.0,2.34913
0,474,53,59.255164,319,28,28.443523,91,9,7.950734,66,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,242,32,32.148408,149,11,17.249557,44,7,4.369843,33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



columns:
situation.OpenPlay.shots
situation.OpenPlay.goals
situation.OpenPlay.xG
situation.OpenPlay.against.shots
situation.OpenPlay.against.goals
situation.OpenPlay.against.xG
situation.FromCorner.shots
situation.FromCorner.goals
situation.FromCorner.xG
situation.FromCorner.against.shots
situation.FromCorner.against.goals
situation.FromCorner.against.xG
situation.DirectFreekick.shots
situation.DirectFreekick.goals
situation.DirectFreekick.xG
situation.DirectFreekick.against.shots
situation.DirectFreekick.against.goals
situation.DirectFreekick.against.xG
situation.SetPiece.shots
situation.SetPiece.goals
situation.SetPiece.xG
situation.SetPiece.against.shots
situation.SetPiece.against.goals
situation.SetPiece.against.xG
situation.Penalty.shots
situation.Penalty.goals
situation.Penalty.xG
situation.Penalty.against.shots
situation.Penalty.against.goals
situation.Penalty.against.xG
formation.4-3-3.stat
formation.4-3-3.time
formation.4-3-3.shots
formation.4-3-3.goals
formation.4-3-3.xG
for

In [ ]:
async with aiohttp.ClientSession() as session:
    understat = Understat(session)
    results = await understat.get_league_results(
        "la_liga",
        2023,
    )

season_la_liga_result = pd.DataFrame(pd.json_normalize(results))
# season_la_liga_result['datetime'] = pd.to_datetime(season_la_liga_result['datetime'])
# season_la_liga_result = season_la_liga_result[season_la_liga_result['datetime'] < pd.Timestamp('2023-10-28')]
# season_la_liga_result = season_la_liga_result[season_la_liga_result['a.title'].isin(['Barcelona', 'Real Madrid']) | season_la_liga_result['h.title'].isin(['Barcelona', 'Real Madrid'])]

#season_la_liga_result.head(5)

In [168]:
shots_df.to_csv('shots.csv')
players_df.to_csv('players.csv')
player_matches_history_df.to_csv('player_matches_history.csv')
player_shots_history_df.to_csv('player_shots_history.csv')
player_avg_stats_df.to_csv('player_avg_stats.csv')
team_res_history_df.to_csv('team_res_history.csv')
team_stats_history_df.to_csv('team_stats_history.csv')
season_la_liga_result.to_csv('season_results.csv')